In [25]:
#!pip install gym
#!pip install tensorflow[and-cuda]
#!pip install tensorflow==2.5.0 keras-rl2
#!pip install keras-rl2
#!pip uninstall tensorflow -y
# not compatible with tf >2.15
#!pip uninstall keras-rl2 -y
#!pip install tensorflow-addons==0.16.1
#!pip uninstall keras
#!pip install pygame

In [3]:
#!pip list

In [4]:
#!pip uninstall tensorflow tensorflow-gpu tensorboard tensorflow-estimator tensorflow-io tensorflow-io-gcs-filesystem -y

#!pip uninstall keras-nightly Keras-Preprocessing -y
#!pip uninstall keras-rl2

In [5]:
#!pip cache purge

In [6]:
#!pip list


In [7]:
#keras.__version__
#!pip uninstall keras -y

In [8]:
#print(tf.sysconfig.get_build_info())

In [9]:
# set tf warning level to 2 ....shows errors but not warnings
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
#tf.__version__

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import os


import tensorflow as tf
#tf.config.run_functions_eagerly(True)
#print(tf.config.functions_run_eagerly())
from tensorflow import keras
from tensorflow.keras.models import load_model


#from rl.agents import DQNAgent
#from rl.policy import BoltzmannQPolicy
#from rl.memory import SequentialMemory


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# tyr legacy adam due to numpy error
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import numpy as np
import random

import pygame

2024-09-23 21:58:32.699121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 21:58:32.714025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 21:58:32.719513: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 21:58:32.732732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 21:58:33.377651: W tensorflow/compiler/tf2

In [2]:
print(tf.sysconfig.get_build_info())

OrderedDict([('cpu_compiler', '/usr/lib/llvm-17/bin/clang'), ('cuda_compute_capabilities', ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90']), ('cuda_version', '12.3'), ('cudnn_version', '8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', True)])


In [3]:
# Check if TensorFlow is built with CUDA support
if tf.config.list_physical_devices('GPU'):
    print("CUDA is available")
    print(f"Device: {tf.config.list_physical_devices('GPU')[0]}")
else:
    print("CUDA is not available")

CUDA is available
Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


I0000 00:00:1727125122.508317   28292 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727125122.564999   28292 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727125122.565278   28292 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


# Create Environment

In [6]:
# inherit from gym env
class SwatEnv(Env):
    def __init__(self):
        print('init reached')
        # 0 decrease, 1 hold, 2 increase
        self.action_space = Discrete(3)
        # temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100] ))
        # set start temp
        self.state = 38 + random.randint(-3, 3)
        # set shower length
        self.shower_length = 60
        
    def step(self, action):
        #print(f'step reached {action}')
        # apply action
        # subtract 1 fro action so 1 - 1 = 0 no change
        self.state = self.state + action -1
        # reduce shower length
        self.shower_length = self.shower_length - 1
        
        # calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        # apply temp noise
        self.state += random.randint(-1,1)
        # place holder for info
        info = {}
        
        return int(self.state), reward, done, info

    def reset(self):
        # reset temp
        self.state = 38 + random.randint(-3, 3)
        # reset shower time
        self.shower_length = 60
        return int(self.state)
        
    #def render(self):
        # for vis ie pygame
        #pass
    def render(self, screen, font):
        screen.fill((255, 255, 255))  # Fill the screen with white

        # Draw the temperature meter
        pygame.draw.rect(screen, (0, 0, 255), (100, 150, 50, 200))  # Blue background
        temp_height = (self.state - 0) / 100 * 200  # Scale temp to meter height
        pygame.draw.rect(screen, (255, 0, 0), (100, 350 - temp_height, 50, temp_height))  # Red temperature

        # Display temperature value
        temp_text = font.render(f'Temp: {self.state}°C', True, (0, 0, 0))
        screen.blit(temp_text, (200, 150))

        # Display shower length
        shower_text = font.render(f'Shower Time: {self.shower_length}s', True, (0, 0, 0))
        screen.blit(shower_text, (200, 200))
    

        


In [7]:

def main():
    # Initialize pygame
    pygame.init()
    screen = pygame.display.set_mode((600, 400))
    pygame.display.set_caption("SWAT Environment Visualization")

    # Font for rendering text
    font = pygame.font.SysFont('Arial', 24)

    # Create the environment
    env = SwatEnv()

    # Initialize game loop variables
    running = True
    clock = pygame.time.Clock()

    state = env.reset()  # Initialize environment state

    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Action logic (can be random for testing purposes)
        action = random.randint(0, 2)  # Replace with your agent's action

        # Perform step
        state, reward, done, info = env.step(action)

        # Render the environment
        env.render(screen, font)

        # Update the display
        pygame.display.flip()

        # End if the shower is done
        if done:
            state = env.reset()

        # Control frame rate (optional)
        clock.tick(30)  # 30 FPS

    pygame.quit()

if __name__ == "__main__":
    main()


init reached


/home/loz/MSc/MSc_Project/venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [28]:
env = SwatEnv()

init reached


In [29]:
# sample from action space
env.action_space.sample()

1

In [30]:
env.observation_space.sample()

array([61.075497], dtype=float32)

In [31]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        print(n_state)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

39
39
40
39
40
42
41
42
42
41
41
41
41
41
43
43
43
44
45
44
45
44
43
44
45
44
44
44
44
42
43
44
44
44
44
44
42
42
42
42
43
44
42
43
43
42
42
40
40
38
38
40
39
38
38
39
37
37
36
36
Episode:1 Score:-40
38
39
39
39
39
39
40
38
38
38
37
36
36
36
36
36
36
34
35
35
37
36
38
36
36
38
38
37
36
36
36
37
36
36
35
35
35
35
35
37
38
38
36
36
36
36
37
35
35
34
33
33
32
32
31
32
30
29
30
29
Episode:2 Score:-6
40
42
42
43
43
41
41
42
42
43
41
40
40
39
39
37
38
40
42
44
43
44
45
46
48
46
46
47
46
46
46
45
44
43
43
43
42
42
41
42
42
43
43
45
46
46
48
49
50
52
52
52
50
49
49
47
47
49
48
50
Episode:3 Score:-52
39
37
39
39
39
41
41
41
41
41
40
41
43
43
43
43
42
40
40
40
39
38
36
35
36
37
36
34
34
35
34
35
36
37
38
40
40
40
41
39
40
40
39
40
39
40
39
38
38
38
38
38
39
38
37
37
38
39
41
40
Episode:4 Score:-8
39
40
40
42
42
42
41
40
40
41
40
42
43
44
44
45
45
45
47
47
49
51
52
53
53
53
55
57
57
58
56
58
56
55
53
53
52
51
52
51
52
51
52
51
49
51
51
53
55
54
53
53
53
53
55
56
57
57
59
60
Episode:5 Score:-58
36

# Create Deep Learning Model#

In [32]:
states = env.observation_space.shape
actions = env.action_space.n

In [33]:
states

(1,)

In [34]:
actions

3

In [35]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    #model.add(Dense(24, activation='relu', input_shape=(1,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [36]:
#del model

In [37]:
model = build_model(states, actions)

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# Build Agent wit Keras-RL

In [39]:
#@tf.function(reduce_retracing=True)
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [40]:
dqn = build_agent(model, actions)

In [41]:
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [42]:


dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)


Training for 5000 steps ...
Interval 1 (0 steps performed)
 1423/10000 [===>..........................] - ETA: 1:36 - reward: -0.5264done, took 16.064 seconds


In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 10.000, steps: 60
Episode 2: reward: 16.000, steps: 60
Episode 3: reward: 6.000, steps: 60
Episode 4: reward: 8.000, steps: 60
Episode 5: reward: 4.000, steps: 60
Episode 6: reward: 18.000, steps: 60
Episode 7: reward: -16.000, steps: 60
Episode 8: reward: -12.000, steps: 60
Episode 9: reward: -8.000, steps: 60
Episode 10: reward: -2.000, steps: 60
Episode 11: reward: -2.000, steps: 60
Episode 12: reward: 8.000, steps: 60
Episode 13: reward: -10.000, steps: 60
Episode 14: reward: 4.000, steps: 60
Episode 15: reward: 2.000, steps: 60
Episode 16: reward: 6.000, steps: 60
Episode 17: reward: -10.000, steps: 60
Episode 18: reward: 0.000, steps: 60
Episode 19: reward: 10.000, steps: 60
Episode 20: reward: -8.000, steps: 60
Episode 21: reward: -6.000, steps: 60
Episode 22: reward: 6.000, steps: 60
Episode 23: reward: 0.000, steps: 60
Episode 24: reward: -2.000, steps: 60
Episode 25: reward: 8.000, steps: 60
Episode 26: reward: 6.000, steps: 60


In [12]:
os.getcwd()

'/home/loz/Documents/GitHub/MSc-Project/final_code'

In [26]:
#os.listdir()

In [24]:
# get pre_trained model of SWaT
swat_lstm = tf.keras.models.load_model('swat_lstm_100s.keras')


OSError: Unable to synchronously open file (file signature not found)